<a href="https://colab.research.google.com/github/DanishJameel/Open_source/blob/main/GDP_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df =pd.read_csv("/content/GDP during pandamic -2020.csv")

FileNotFoundError: ignored

In [ ]:
df

In [ ]:
# Dealing with outliers
# Here, we assume the column 'GDP_growth_percentage' contains outliers and use a simple approach to remove them
# Adjust the threshold value according to your data and requirements
threshold = 3  # Example threshold for identifying outliers
df = df[abs(df['GDP_growth_percentage'] - df['GDP_growth_percentage'].mean()) < threshold]

# Print the dataset after dealing with outliers
print(df)

In [ ]:
import pandas as pd

# Handling missing data
df.dropna(inplace=True)  # Remove rows with missing values

# Dealing with outliers
# Here, we assume the column 'GDP_growth_percentage' contains outliers and use a simple approach to remove them
# Adjust the threshold value according to your data and requirements
threshold = 3  # Example threshold for identifying outliers
df = df[abs(df['GDP_growth_percentage'] - df['GDP_growth_percentage'].mean()) < threshold]

# Transforming data
# If 'Rise_fall_GDP' column contains text categories, we can map them to numerical values
category_mapping = {
    'medium fall': -1,
    'high fall': -2,
    'very high fall': -3
}
df['Rise_fall_GDP'] = df['Rise_fall_GDP'].map(category_mapping)

# Print the cleaned and processed dataset
print(df)

In [ ]:
# Define the column name for which you want to remove outliers
column_name = 'GDP_growth_percentage'

# Calculate the quartiles and interquartile range (IQR)
Q1 = df[column_name].quantile(0.25)
Q3 = df[column_name].quantile(0.75)
IQR = Q3 - Q1

# Set the threshold for identifying outliers
threshold = 1.5

# Remove rows with outliers
df = df[~((df[column_name] < (Q1 - threshold * IQR)) | (df[column_name] > (Q3 + threshold * IQR)))]

# Print the dataset after removing outliers
print(df)

In [ ]:

# Compute z-scores for the 'GDP_growth_percentage' column
z_scores = zscore(df['GDP_growth_percentage'])

# Set a threshold for identifying outliers
threshold = 3

# Remove rows with outliers
df = df[(np.abs(z_scores) < threshold)]

# Print the dataset after removing outliers
print(df)

In [ ]:
# Generate descriptive statistics
print(df.describe())


In [ ]:
# Example: Line plot of GDP growth percentage over time
plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='Country', y='GDP_growth_percentage')
plt.title('GDP Growth Percentage')
plt.xlabel('Country')
plt.ylabel('GDP Growth Percentage')

# Reduce the gap between X-axis points
plt.xticks(rotation=90)
plt.tight_layout()  # Adjust spacing between plot elements

plt.show()

In [ ]:

# Create a boxplot of 'GDP_growth_percentage' using seaborn
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, y='GDP_growth_percentage')
plt.title('Boxplot of GDP Growth Percentage')
plt.ylabel('GDP Growth Percentage')
plt.show()

In [ ]:

# Convert 'Nominal_gdp_p_capita' column to numeric
df['Nominal_gdp_p_capita'] = pd.to_numeric(df['Nominal_gdp_p_capita'], errors='coerce')

# Select the top 20 values from 'Nominal_gdp_p_capita'
top_20 = df['Nominal_gdp_p_capita'].nlargest(20)

# Create a histogram of the top 20 values
plt.figure(figsize=(25, 6))
top_20.hist(bins=10)
plt.title('Histogram of Top 20 Nominal GDP per capita')
plt.xlabel('Nominal GDP per capita')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Create a scatterplot of 'Nominal_gdp_p_capita' vs. 'PPP_gdp_p_capita'
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='Nominal_gdp_p_capita', y='PPP_gdp_p_capita')
plt.title('Scatterplot: Nominal GDP per capita vs. PPP GDP per capita')
plt.xlabel('Nominal GDP per capita')
plt.ylabel('PPP GDP per capita')
plt.show()

# Create a heatmap of correlation between variables
correlation_matrix = df[['Nominal_gdp_p_capita', 'PPP_gdp_p_capita', 'GDP_growth_percentage']].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
import pandas as pd
import scipy.stats as stats


# Perform a hypothesis test to compare the mean GDP growth percentage of two groups
group1 = df[df['Rise_fall_GDP'] == 'medium fall']['GDP_growth_percentage']
group2 = df[df['Rise_fall_GDP'] == 'very high fall']['GDP_growth_percentage']

# Perform an independent t-test
t_statistic, p_value = stats.ttest_ind(group1, group2)

# Check the p-value to determine the significance of the difference
alpha = 0.05  # Set the significance level
if p_value < alpha:
    conclusion = "There is a significant difference in mean GDP growth percentage between the two groups."
else:
    conclusion = "There is no significant difference in mean GDP growth percentage between the two groups."

# Print the results
print("Independent T-Test Results:")
print("Group 1 Mean:", group1.mean())
print("Group 2 Mean:", group2.mean())
print("T-Statistic:", t_statistic)
print("P-Value:", p_value)
print("Conclusion:", conclusion)

In [ ]:
!pip install dash

In [ ]:
!pip install jupyter-dash
!pip install plotly==5.5.0
!pip install pandas


In [ ]:
import pandas as pd
import jupyter_dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
app = jupyter_dash.JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Data Exploration Dashboard"),
    html.Div([
        html.Label("Select a variable to visualize:"),
        dcc.Dropdown(
            id='variable-dropdown',
            options=[
                {'label': 'GDP Growth Percentage', 'value': 'GDP_growth_percentage'},
                {'label': 'Nominal GDP per capita', 'value': 'Nominal_gdp_p_capita'},
                {'label': 'PPP GDP per capita', 'value': 'PPP_gdp_p_capita'},
                # Add more options for other variables
            ],
            value='GDP_growth_percentage'
        )
    ], style={'width': '300px', 'margin-bottom': '20px'}),
    dcc.Graph(id='visualization-graph'),
])
@app.callback(
    Output('visualization-graph', 'figure'),
    [Input('variable-dropdown', 'value')]
)
def update_graph(selected_variable):
    fig = px.line(df, x='Country', y=selected_variable)
    fig.update_layout(title=f'{selected_variable} over Time', xaxis_title='Country', yaxis_title=selected_variable)
    return fig
if __name__ == '__main__':
    app.run_server(mode='inline')



In [ ]:
!pip install pyppeteer
